In [4]:
import json 

from dataflow.core.lispress import parse_lispress, render_compact, lispress_to_program

In [5]:

data = [json.loads(x) for x in open("/brtx/603-nvme1/estengel/calflow_calibration/tune_roberta_number_tokenize/translate_output_calibrated/calibration.tgt").readlines()]

In [6]:
data_by_src_str = {x['src_str']: [] for x in data}
for x in data: 
    data_by_src_str[x['src_str']].append(x)

In [ ]:
from typing import List
import numpy as np

from dataflow.core.program import Expression

class Lattice:
    def __init__(self,
                expressions: List[List[Expression]],
                probs: List[List[float]]
        ) -> None:
        self.expressions = expressions
        self.probs = probs 

        self.lattice = self.build_lattice()

    @property
    def max_expression_length(self):
        return max([len(x) for x in self.expressions])

    def build_lattice(self):
        # build lattice of expressions; last dim is prob and pointer to expression id 
        lattice = np.zeros((len(self.expressions), self.max_expression_length, 2))

        

In [7]:

def get_confident_prefix(list_of_outputs, threshold = 0.30):
    prefix = []
    options = []
    for output in list_of_outputs:
        program, __ = lispress_to_program(parse_lispress(output['tgt_str']),0)
        expressions = program.expressions
        expressions.reverse()
        probs = output['expression_probs']
        for i, (e, p) in enumerate(zip(expressions, probs)):
            if p > threshold:
                prefix.append(e)
            else:
                options.append(expressions[i:]) 
    return prefix, options


datapoint = data_by_src_str['__User create a meeting 0907 tomorrow']
prefix, options = get_confident_prefix(datapoint)



In [8]:
import sys

sys.path.insert(0, "/home/estengel/semantic_parsing_with_constrained_lm/src")

# from semantic_parsing_with_constrained_lm.
from semantic_parsing_with_constrained_lm.scfg.generate import parse_and_render
from semantic_parsing_with_constrained_lm.scfg.scfg import SCFG


In [ ]:
scfg = SCFG.from_folder(str("/home/estengel/semantic_parsing_with_constrained_lm/src/semantic_parsing_with_constrained_lm/domains/calflow/grammar/"))


In [10]:
def canonicalize(lispress, scfg):
    lispress = render_compact(parse_lispress(lispress))
    print(lispress)
    canonical_utterances = parse_and_render(
        scfg, " " + lispress, source_is_plan=True
    )
    return canonical_utterances

lispress = datapoint[0]['tgt_str']
print(canonicalize(lispress, scfg))

(Yield (CreateCommitEventWrapper (CreatePreflightEventWrapper (Event.start_? (?= (DateAtTimeWithDefaults (Tomorrow) (HourMilitary 8L)))))))
At """ (Yield ^""",
expected one of [':'],
but got "(".
